In [1]:
import glob
import zipfile
import json
import pandas as pd
import pickle
from pprint import pprint

In [2]:
data_path = "/home/singh_shruti/data/mag_aminer/"

## Count references in the whole dataset

In [ ]:
with open('mag_papers.pickle', 'rb') as handle:
    all_mag_papers = pickle.load(handle)

In [ ]:
total_papers = 0
total_refs = 0

for v in all_mag_papers.values():
    total_papers += 1
    if "references" in v and v["references"]:
        total_refs += 1

## Filtering subsests

In [3]:
paper_files = glob.glob(data_path + "mag_papers_[0-9].zip")
author_files = glob.glob(data_path + "mag_authors_[0-9].zip")

In [ ]:
mag_papers = {}

later_2016 = 0
venue_missing = 0
till_2016 = 0
references_present = 0

for f in paper_files:
    archive = zipfile.ZipFile(f, 'r')
    zip_files = archive.namelist()
    print(zip_files)
    for fi in zip_files:
        bytes_text = archive.read(fi)
        print("Read file {} in archive {}".format(fi, zip_files))
        try:
            str_list = bytes_text.decode('utf-8').split("\n")
            print("Length of file: ", len(str_list))
            idx = 0
            
            for l in str_list:
                if idx % 1000000 == 0:
                    print(idx)
                idx += 1
                try:
                    paper_dict = json.loads(l)
                    if "year" in paper_dict and paper_dict["year"]:
                        if paper_dict["year"] < 2017:
                            till_2016 = till_2016 + 1
                            mag_papers[paper_dict["id"]] = paper_dict
                            if "references" in paper_dict and paper_dict["references"]:
                                references_present += 1
                        else:
                            later_2016 = later_2016 + 1
                    else:
                        venue_missing = venue_missing + 1
                except Exception as ex:
                    print("err for json loading")
                    print(ex)
        except Exception as ex:
            print("err for bytes decoding")
            print(ex)

In [ ]:
with open('subset_mag_papers_2016.pickle', 'wb') as handle:
    pickle.dump(mag_papers, handle)